# Homework 1: Predicting Duration of a Ride

We'll use "Yellow Taxi Trip Records" to train the prediction model - [https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page]

In [ ]:
## importing necessary libraries
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import root_mean_squared_error 

In [3]:
## creating a dataframe from the .parquet file for January 2023 data
df = pd.read_parquet("/workspaces/mlops-practice/yellow_tripdata_2023-01.parquet")

In [4]:
## checking number of columns in the dataset and printing the output
print("Number of columns:", df.shape[1])

Number of columns: 19


**Number of columns: 19**

In [5]:
# creating a separate column, the duration in minutes
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

In [6]:
#checking std. deviation in our data
std_duration = df['duration'].std()
print('std_duration is equal to:', round(std_duration, 2))

std_duration is equal to: 42.59


**The standard deviation of trip duration is equal to 42.59**

In [7]:
counter_before = len(df)
df_filtered = df[(df['duration'] >= 1) & (df['duration'] <= 60)]
remaining_fraction = len(df_filtered) / counter_before

print(remaining_fraction)


0.9812202822125979


**98% are left after dropping outliers**

In [8]:
## one-hot encoding piece
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']


In [9]:
df_filtered[categorical] = df_filtered[categorical].astype(str)
dicts = df_filtered[categorical + numerical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(dicts)

/tmp/ipykernel_31134/1160693391.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[categorical] = df_filtered[categorical].astype(str)


In [10]:
print(X_train.shape)

(3009173, 516)


**The dimensionality of the matrix is 516**

In [11]:
target = 'duration'
y_train = df_filtered[target].values

In [12]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_train)
root_mean_squared_error(y_train, y_pred)

7.658396898909143

In [3]:
df_val = pd.read_parquet('/workspaces/mlops-practice/yellow_tripdata_2023-02.parquet')
df_val['duration'] = (df_val['tpep_dropoff_datetime'] - df_val['tpep_pickup_datetime']).dt.total_seconds() / 60

In [4]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
df_val = df_val[(df_val['duration'] >= 1) & (df_val['duration'] <= 60)]
df_val[categorical] = df_val[categorical].astype(str)

In [5]:
train_dicts = df_val[categorical + numerical].to_dict(orient='records')
dv_val = DictVectorizer()
X_train_val = dv_val.fit_transform(train_dicts)

In [7]:
target_val = 'duration'
y_train_val = df_val[target_val].values

In [ ]:
lr2 = LinearRegression()
lr2.fit(X_train_val, y_train_val)
y_pred_2 = lr2.predict(X_train_val)

In [9]:
root_mean_squared_error(y_train_val,y_pred_2)

7.787924973977241